In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir("../")
os.getcwd()

'c:\\workspace\\EagSession1\\web_agent'

In [3]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv, find_dotenv

# Load environment variables
load_dotenv(find_dotenv())

# Import required modules
from langchain_openai import AzureChatOpenAI

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    temperature=0
)


# Create a prompt template for math questions.
math_prompt = ChatPromptTemplate.from_template(
    """
You are a helpful math tutor. Please solve the following math problem step by step:

Question: {math_question}

Please show your work and explain each step clearly.
"""
)

# Create the chain.
math_chain = LLMChain(llm=llm, prompt=math_prompt, verbose=True)

# Example usage.
question = "What is the area of a circle with radius 5 units?"
response = math_chain.invoke({"math_question": question})
print(response)

# Try another example.
question2 = "Solve the equation: 2x + 5 = 15"
response2 = math_chain.invoke({"math_question": question2})
print(response2)

c:\Users\40104002\AppData\Local\anaconda3\envs\ic-engine\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Human: 
You are a helpful math tutor. Please solve the following math problem step by step:

Question: What is the area of a circle with radius 5 units?

Please show your work and explain each step clearly.


> Finished chain.
{'math_question': 'What is the area of a circle with radius 5 units?', 'text': "To find the area of a circle, we use the formula:\n\n\\[ \\text{Area} = \\pi r^2 \\]\n\nwhere \\( r \\) is the radius of the circle and \\( \\pi \\) is a mathematical constant approximately equal to 3.14159.\n\nLet's solve the problem step by step:\n\n1. **Identify the radius**: The problem states that the radius \\( r \\) of the circle is 5 units.\n\n2. **Substitute the radius into the formula**: Replace \\( r \\) in the formula with 5.\n\n   \\[ \\text{Area} = \\pi \\times (5)^2 \\]\n\n3. **Calculate \\( r^2 \\)**: Compute the square of the radius.\n\n   \\[ 5^2 = 25 \\]\n\n4. **Substitute \\( r^2 \\) back into the formula*

In [23]:
import os
import math
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain.tools import StructuredTool
import os
import math
from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

In [26]:


# Define Pydantic models for input validation
class CircleAreaInput(BaseModel):
    """Input schema for circle area calculation."""
    radius: float = Field(..., description="The radius of the circle.")

class LinearEquationInput(BaseModel):
    """Input schema for linear equation solving."""
    equation: str = Field(..., description="A linear equation of the form 'ax + b = c'.")

# Update function signatures to match the field names of the Pydantic models.
def calculate_circle_area(radius: float) -> str:
    """Calculate the area of a circle given its radius.

    Args:
        radius: The radius of the circle in units
    Returns:
        The area of the circle in square units
    """
    area = math.pi * radius ** 2
    return f"The area of a circle with radius {radius} is {area:.2f} square units."

def solve_linear_equation(equation: str) -> str:
    """
    Solve a simple linear equation of the form 'ax + b = c'.
    This function assumes the equation is properly formatted.

    Args:
        equation: A linear equation of the form 'ax + b = c'
    Returns:
        The solution for x
    """
    try:
        eq = equation.replace(" ", "")
        x_index = eq.index("x")
        a = float(eq[:x_index])
        remainder = eq[x_index + 1:]
        if remainder[0] in "+-":
            sign = 1 if remainder[0] == "+" else -1
            remainder = remainder[1:]
        else:
            sign = 1
        b_str, c_str = remainder.split("=")
        b = sign * float(b_str)
        c = float(c_str)
        x = (c - b) / a
        return f"The solution for the equation {equation} is x = {x}."
    except Exception as e:
        return f"Error parsing the equation '{equation}': {str(e)}"

# Wrap the functions as StructuredTools using the Pydantic input schemas.
circle_area_tool = StructuredTool.from_function(
    func=calculate_circle_area,
    input_schema=CircleAreaInput,
    name="CalculateCircleArea",
    description="Calculate the area of a circle given its radius. Input: a JSON with a 'radius' key."
)

linear_equation_tool = StructuredTool.from_function(
    func=solve_linear_equation,
    input_schema=LinearEquationInput,
    name="SolveLinearEquation",
    description="Solve a linear equation of the form 'ax + b = c'. Input: a JSON with an 'equation' key."
)

tools = [circle_area_tool, linear_equation_tool]

In [27]:
# Bind tools to the model
llm_with_tools = llm.bind_tools(tools)

# Create new prompt template for tool calling
prompt = """You are a helpful AI assistant that uses available tools/functions to solve math problems.
When given a math problem, analyze it carefully and use the appropriate function to solve it.
For circle area calculations, use the calculate_circle_area tool.
For linear equations, use the solve_linear_equation tool.
Always use the tools instead of doing calculations yourself.

Question: {input}
"""

# Test with the same queries
query1 = "What is the area of a circle with radius 5 units?"
result1 = llm_with_tools.invoke(prompt.format(input=query1))
print("\nResult 1:", result1)

query2 = "Solve the equation: 2x + 5 = 15"
result2 = llm_with_tools.invoke(prompt.format(input=query2))
print("\nResult 2:", result2)


Result 1: content='' additional_kwargs={'tool_calls': [{'id': 'call_tn0t46Kkx1H3mNdK9f5eMo8v', 'function': {'arguments': '{"radius":5}', 'name': 'CalculateCircleArea'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 183, 'total_tokens': 199, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ded0d14823', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}} id='run-8c5d0999-dc57-4a69-be1a-c59fb4ffe09b-0' tool_calls=[{'n

In [33]:
result1.tool_calls[0]

{'name': 'CalculateCircleArea',
 'args': {'radius': 5},
 'id': 'call_tn0t46Kkx1H3mNdK9f5eMo8v',
 'type': 'tool_call'}

In [34]:
def execute_tool_calls(tools_dict: dict, model_output) -> str:
    """
    Execute tool calls based on the model's output.

    Args:
        tools_dict: Dictionary mapping tool names to tool objects
        model_output: Output from the LLM model containing tool calls

    Returns:
        str: Result of executing the tool calls
    """
    if not hasattr(model_output, "tool_calls") or not model_output.tool_calls:
        return "No tool calls found in model output"

    results = []
    for tool_call in model_output.tool_calls:
        tool_name = tool_call['name']
        if tool_name not in tools_dict:
            results.append(f"Tool {tool_name} not found in available tools")
            continue

        tool = tools_dict[tool_name]
        try:
            result = tool.invoke(tool_call['args'])
            results.append(result)
        except Exception as e:
            results.append(f"Error executing {tool_name}: {str(e)}")

    return "\n".join(results)


# Create a dictionary mapping tool names to tool objects
tools_dict = {
    "CalculateCircleArea": circle_area_tool,
    "SolveLinearEquation": linear_equation_tool,
}

# Test with the previous results
print("\nExecuting tool calls for Result 1:")
print(execute_tool_calls(tools_dict, result1))

print("\nExecuting tool calls for Result 2:")
print(execute_tool_calls(tools_dict, result2))


Executing tool calls for Result 1:
The area of a circle with radius 5.0 is 78.54 square units.

Executing tool calls for Result 2:
The solution for the equation 2x + 5 = 15 is x = 5.0.
